<a href="https://colab.research.google.com/github/harshelke180502/Big_Data_Mining-CS483-Harsh_Shelke/blob/main/Harsh_Shelke_Colab_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CS246 - Colab 3
## K-Means & PCA

### Setup

Let's set up Spark on your Colab environment.  Run the cell below!

In [1]:
!pip install pyspark==3.5.1
!pip install -U -q PyDrive
# !apt install openjdk-8-jdk-headless -qq
!apt-get update -qq
!apt-get install openjdk-11-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"

import pyspark
pyspark.__version__


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


'3.5.1'

Now we import some of the libraries usually needed by our workload.





In [2]:
!pip install numpy matplotlib

!pip install pandas==2.2.2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

print("Spark:", pyspark.__version__)
print("Pandas:", pd.__version__)

Spark: 3.5.1
Pandas: 2.2.2


Let's initialize the Spark context.

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("CS246-Colab").config("spark.ui.port", "4050").getOrCreate()

sc = spark.sparkContext

### Data Preprocessing

In this Colab, rather than downloading a file from Google Drive, we will load a famous machine learning dataset, the [Breast Cancer Wisconsin dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_breast_cancer.html), using the ```scikit-learn``` datasets loader.

In [4]:
from sklearn.datasets import load_breast_cancer
breast_cancer = load_breast_cancer()

For convenience, given that the dataset is small, we first

*   construct a Pandas dataframe
*   tune the schema
*   and convert it into a Spark dataframe.

In [5]:
pd_df = pd.DataFrame(breast_cancer.data, columns=breast_cancer.feature_names)

df = spark.createDataFrame(pd_df)

def set_df_columns_nullable(spark, df, column_list, nullable=False):
    for struct_field in df.schema:
        if struct_field.name in column_list:
            struct_field.nullable = nullable
    df_mod = spark.createDataFrame(df.rdd, df.schema)
    return df_mod

df = set_df_columns_nullable(spark, df, df.columns)
df = df.withColumn('features', array(df.columns))
vectors = df.rdd.map(lambda row: Vectors.dense(row.features))

df.printSchema()

root
 |-- mean radius: double (nullable = false)
 |-- mean texture: double (nullable = false)
 |-- mean perimeter: double (nullable = false)
 |-- mean area: double (nullable = false)
 |-- mean smoothness: double (nullable = false)
 |-- mean compactness: double (nullable = false)
 |-- mean concavity: double (nullable = false)
 |-- mean concave points: double (nullable = false)
 |-- mean symmetry: double (nullable = false)
 |-- mean fractal dimension: double (nullable = false)
 |-- radius error: double (nullable = false)
 |-- texture error: double (nullable = false)
 |-- perimeter error: double (nullable = false)
 |-- area error: double (nullable = false)
 |-- smoothness error: double (nullable = false)
 |-- compactness error: double (nullable = false)
 |-- concavity error: double (nullable = false)
 |-- concave points error: double (nullable = false)
 |-- symmetry error: double (nullable = false)
 |-- fractal dimension error: double (nullable = false)
 |-- worst radius: double (nullable

With the next cell, we build the two data structures that we will be using throughout this Colab:


*   ```features```, a dataframe of Dense vectors, containing all the original features in the dataset;
*   ```labels```, a series of binary labels indicating if the corresponding set of features belongs to a subject with breast cancer, or not.



In [6]:
from pyspark.ml.linalg import Vectors
features = spark.createDataFrame(vectors.map(Row), ["features"])
labels = pd.Series(breast_cancer.target)

### Your task

If you run successfully the Setup and Data Preprocessing stages, you are now ready to cluster the data with the [K-means](https://spark.apache.org/docs/latest/ml-clustering.html) algorithm included in MLlib (Spark's Machine Learning library).
Set the ```k``` parameter to **2**, fit the model, and the compute the [Silhouette score](https://en.wikipedia.org/wiki/Silhouette_(clustering)) (i.e., a measure of quality of the obtained clustering).  

**IMPORTANT:** use the MLlib implementation of the Silhouette score (via ```ClusteringEvaluator```).

In [7]:
''' 8-9 lines of code in total expected but can differ based on your style.
for sub-parts of the question, creating different cells of code would be recommended.'''
# YOUR CODE HERE

from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator


kmeans = KMeans(k=2, seed=42, featuresCol="features", predictionCol="prediction")
model = kmeans.fit(features)


predictions = model.transform(features)

# Computed the Silhouette score
evaluator = ClusteringEvaluator(featuresCol="features", predictionCol="prediction")
silhouette = evaluator.evaluate(predictions)

print("Silhouette Score:", silhouette)

Silhouette Score: 0.8342904262826145


Take the predictions produced by K-means, and compare them with the ```labels``` variable (i.e., the ground truth from our dataset).  

Compute how many data points in the dataset have been clustered correctly (i.e., positive cases in one cluster, negative cases in the other).

*HINT*: you can use ```np.count_nonzero(series_a == series_b)``` to quickly compute the element-wise comparison of two series.

**IMPORTANT**: K-means is a clustering algorithm, so it will not output a label for each data point, but just a cluster identifier!  As such, label ```0``` does not necessarily match the cluster identifier ```0```.


In [8]:
''' 4 lines of code in total expected but can differ based on your style. for sub-parts of the question, creating different cells of code would be recommended.'''
# YOUR CODE HERE

import builtins

# Collect predictions
predicted = np.array(predictions.select("prediction").rdd.map(lambda x: x[0]).collect())
true_labels = labels.values

accuracy1 = np.count_nonzero(predicted == true_labels)

# Flipped match (because cluster IDs are arbitrary)
accuracy2 = np.count_nonzero((1 - predicted) == true_labels)

correct = builtins.max(accuracy1, accuracy2)

print("Correctly clustered points:", correct)
print("Total points:", len(true_labels))
print("Accuracy:", correct / len(true_labels))


Correctly clustered points: 486
Total points: 569
Accuracy: 0.8541300527240774


Now perform dimensionality reduction on the ```features``` using the [PCA](https://spark.apache.org/docs/latest/ml-features.html#pca) statistical procedure, available as well in MLlib.

Set the ```k``` parameter to **2**, effectively reducing the dataset size of a **15X** factor.

In [9]:
''' 6 lines of code in total expected but can differ based on your style. for sub-parts of the question, creating different cells of code would be recommended.'''
# YOUR CODE HERE
from pyspark.ml.feature import PCA

# Applied PCA and reduced dimensions to 2
pca = PCA(k=2, inputCol="features", outputCol="pcaFeatures")
pca_model = pca.fit(features)


pca_result = pca_model.transform(features)


pca_features = pca_result.select("pcaFeatures")

pca_features.show(5, truncate=False)


+----------------------------------------+
|pcaFeatures                             |
+----------------------------------------+
|[-2260.013886292542,-187.96030122263687]|
|[-2368.9937557820544,121.58742425815537]|
|[-2095.6652015478608,145.11398565870124]|
|[-692.6905100570509,38.57692259208171]  |
|[-2030.2124927427067,295.2979839927931] |
+----------------------------------------+
only showing top 5 rows



Now run K-means with the same parameters as above, but on the ```pcaFeatures``` produced by the PCA reduction you just executed.

Compute the Silhouette score, as well as the number of data points that have been clustered correctly.

In [10]:
''' 11-13 lines of code in total expected but can differ based on your style. for sub-parts of the question, creating different cells of code would be recommended.'''
# YOUR CODE HERE

from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
import numpy as np
import builtins


kmeans_pca = KMeans(k=2, seed=42,
                    featuresCol="pcaFeatures",
                    predictionCol="prediction")

model_pca = kmeans_pca.fit(pca_features)

predictions_pca = model_pca.transform(pca_features)

# Silhouette score
evaluator_pca = ClusteringEvaluator(featuresCol="pcaFeatures",
                                    predictionCol="prediction")

silhouette_pca = evaluator_pca.evaluate(predictions_pca)

print("PCA Silhouette Score:", silhouette_pca)

# predictions array
predicted_pca = np.array(
    predictions_pca.select("prediction")
                   .rdd.map(lambda x: x[0])
                   .collect()
)

true_labels = labels.values


accuracy1 = np.count_nonzero(predicted_pca == true_labels)

accuracy2 = np.count_nonzero((1 - predicted_pca) == true_labels)

correct_pca = builtins.max(accuracy1, accuracy2)

print("Correctly clustered (PCA):", correct_pca)
print("Accuracy (PCA):", correct_pca / len(true_labels))

PCA Silhouette Score: 0.8348610363444832
Correctly clustered (PCA): 486
Accuracy (PCA): 0.8541300527240774


Once you obtained the desired results, **head over to Gradescope and submit your solution for this Colab**!